# Use the 3d camera
By default the viewer will try to set the ViewProjection matrix on your shader for you, so you can navigate the view in 3d if you want.

* By default the matrix will be sent as row major to the shader, but if you prefer you can work in column major.
    * use : <code>w = ipywebgl.GLViewer(shader_matrix_major='column_major')</code> to be in column major.
    * all the examples will be 'row_major' in this case.

### Import

In [1]:
import ipywebgl
import numpy as np

### Create viewer
And clear the color to make the background visible.

In [2]:
w = ipywebgl.GLViewer()
w.clear_color(.8, .8, .8 ,1)
w.clear()
w.render()
w

GLViewer(camera_pos=[0, 50, 200])

### Program
Create a program that will display a grid on the ground. And use the Camera (ViewProjection) to rotate the view

In [3]:
pr = w.create_program()
pr.compile(
"""#version 300 es

uniform mat4 ViewProjection;
in vec3 in_vert;

void main() {
    gl_Position = ViewProjection * vec4(in_vert, 1.0);
}
"""
,
"""#version 300 es
precision highp float;
out vec4 f_color;
void main() {
    f_color = vec4(0.1, 0.1, 0.1, 1.0);
}
"""
)

### Grid vertex buffer
Create a grid vertex array. By creating a set of lines horizontal and vertical.

In [4]:
def grid(size, steps):
    u = np.repeat(np.linspace(-size, size, steps), 2)
    v = np.tile([-size, size], steps)
    w = np.zeros(steps * 2)
    return np.concatenate([np.dstack([u, w, v]), np.dstack([v, w, u])])

points = grid(100,20)

buf = w.create_buffer()
buf.update(src_data=np.array(
    [ 
        points.flatten()
    ], dtype=np.float32))
vao = w.create_vertex_array()
vao.bind(
    pr, [
        (buf, '3f32', 'in_vert')
    ]
)

In [5]:
#number of points in the buffer :
points.reshape([-1,3]).shape

(80, 3)

### Render
Render the view, you can now interact with the view with your mouse (left click and drag) to rotate and the 'wasd' keys to move

* if you need to change the default keys:
    * you can do it at init time with : <code>GLViewer(move_keys='ijkl')</code>
    * or runtime with : <code>w.move_keys = 'ijkl'</code>

Any letters can be used.

In [6]:
w.clear()
w.use_program(pr)
w.bind_vertex_array(vao)
w.draw_arrays('lines', 0, points.reshape([-1,3]).shape[0])
w.render()
w

GLViewer(camera_pos=[0, 50, 200])

### Interact with the camera from python
Let's set the camera values using the python code

We can build a simple rotating camera control with an interact slider

In [7]:
w.camera_pos, w.camera_pitch, w.camera_yaw

([0, 50, 200], 0.0, 0.0)

In [8]:
from ipywidgets import widgets, interact

def set_camera(angle):
    rad = angle/180.0*np.pi
    w.camera_pos = [np.sin(rad)*200, 70, np.cos(rad)*200]
    w.camera_yaw = angle
    w.camera_pitch = -20
    
interact(
    set_camera, 
    angle=widgets.IntSlider(min=0, max=360, step=1, value=0),
)
w

interactive(children=(IntSlider(value=0, description='angle', max=360), Output()), _dom_classes=('widget-inter…

GLViewer(camera_pitch=-20.0, camera_pos=[0.0, 70, 200.0])